# Simulating noise on Amazon Braket

In [2]:
# Use Braket SDK Cost Tracking to estimate the cost to run this example
from braket.tracking import Tracker
t = Tracker().start()

This notebook gives a detailed overview of noise simulations on Amazon Braket. Amazon Braket provides two noise simulators: a local noise simulator that you can use for free as part of the Braket SDK and an on-demand, high-performing noise simulator, DM1. Both simulators are based on the density matrix formalism. After this tutorial, you will be able to define noise channels, apply noise to new or existing circuits, and run those circuits on the Braket noise simulators. 

### Table of contents:
* [Background](#Background)
    * [Noise simulation based on the density matrix formalism](#density_matrix)
    * [Quantum channel and Kraus representation](#quantum_channel)
* [General imports](#imports)
* [Quick start](#start)
* [Defining noise channels](#noise_channels)
    * [Pre-defined noise channels](#pre-defined)
    * [Defining custom noise channels](#self-defined)
* [Adding noise to a circuit](#apply_noise)
    * [Build noisy circuits bottom-up](#apply_noise_directly)
    * [Applying noise to existing circuits with global methods](#apply_noise_globally)
        * [Applying gate noise to the circuit](#gate-noise)
        * [Applying initialization noise to the circuit](#initialization-noise)
        * [Applying readout noise to the circuit](#readout-noise)
    * [Using both the direct and global methods to apply noise](#both)
* [Running a noisy circuit](#run)

## Background <a class="anchor" id="Background"></a>

### Noise simulation based on the density matrix formalism <a class="anchor" id="density_matrix"></a>
In an ideal case, a quantum state prepared by a noise-free circuit can be described by a state vector $|\psi\rangle$ -- we call it a 'pure state'. However, the presence of noise in realistic quantum devices will introduce classical uncertainty to the quantum state. For example, a bit flip error with 50% probability acting on a qubit flips the $|0\rangle$ state into either $|0\rangle$ or $|1\rangle$ with a 50-50 chance. Note that this is different from an Hadamard-gate acting on $|0\rangle$: The latter results in a coherent superposition of $|0\rangle$ and $|1\rangle$, whereas the former is a classical, so-called mixture of $|0\rangle$ and $|1\rangle$. The most general way of describing a quantum state in the presence of noise is through the so-called density matrix: $\rho = \sum_i p_i|\psi_i\rangle\langle\psi_i|$. It can be understood as a classical mixture of a series of pure states $|\psi_i\rangle$ (each of which could be highly entangled), where $p_i$ is the probability of the state being in $|\psi_i\rangle$. Because the $p_i$ are classical probabilities they have to sum up to 1: $\sum_i p_i = 1$. The density matrix of a pure state is simply $\rho = |\psi\rangle\langle\psi|$ and, in the bit-flip example from above, the density matrix would be $\rho = 0.5|0\rangle\langle 0| + 0.5|1\rangle\langle 1|$. 

The density matrix formalism is a very useful way to describe a noisy system with probabilistic outcomes. It gives an exact description of a quantum system going through a quantum channel with noise. Besides, the expectation value of an observable $\langle O\rangle$ can be easily calculated by $\rm{Tr}(O\rho)$, where "$\rm{Tr}$" is the trace operator. 

### Quantum channel and Kraus representation <a class="anchor" id="quantum_channel"></a>

A [quantum channel](https://en.wikipedia.org/wiki/Quantum_channel) describes the time evolution of a quantum state which is expressed as a density matrix. For instance, to understand what a series of noisy gates does to the state of a quantum computer, you can apply a quantum channel corresponding to the different gate and noise operations. 
Mathematically speaking, a quantum channel is a completely positive and trace-preserving (CPTP) linear map acting on a density matrix. Completely positive means the channel maps positive operators into positive operators (even if the operator is applied to part of a larger system) to make sure the density matrix describes a proper quantum state after the map. Trace-preserving means the trace of the density matrix remains unchanged during the mapping process (this is so that after the map the classical probabilities $p_i$ still sum to 1). 

The so-called _Kraus representation_ is a commonly used representation for CPTP maps. [Kraus's theorem](https://en.wikipedia.org/wiki/Quantum_operation#Kraus_operators) states that any quantum operation acting on a quantum state $\rho$ can be expressed as a map $\varepsilon(\rho) = \sum_i K_i\rho K_i^{\dagger}$, and it satisfies: $\sum_i K_i^{\dagger}K_i = \mathbb{1}$, where $\mathbb{1}$ is the Identity operator.

Let's get started and have a look how you can define and simulate noisy circuits on Amazon Braket.

## General imports <a class="anchor" id="imports"></a>

Let's begin with the usual imports.

In [3]:
from braket.aws import AwsDevice
from braket.circuits import Circuit, gates, noises, observables
from braket.devices import LocalSimulator
from braket.parametric import FreeParameter
import numpy as np
from scipy.stats import unitary_group
import math

## Noise Simulator from AWS

In [27]:
from braket.circuits.noise_model import (
    GateCriteria,
    NoiseModel,
    ObservableCriteria,
)
from braket.circuits import Circuit, Observable, Gate
from braket.circuits.noises import (
    BitFlip,
    Depolarizing,
    TwoQubitDepolarizing,
)
from braket.devices import LocalSimulator
import numpy as np
import math

def noise_model():
    rng = np.random.default_rng()
    m = NoiseModel()
    
    two_q_depo_mu = 1 - 0.9311
    two_q_depo_sigma = 0.005
    bf_mu = 1 - 0.99752
    bf_sigma = 0.0015
    one_q_depo_mu = 1 - 0.9981
    one_q_depo_sigma = 0.00017
    for qi in range(11):
        z_bf_prob = bf_mu + bf_sigma * rng.standard_normal()
        z_bf_prob = 0.0 if z_bf_prob < 0.0 else z_bf_prob
        
        bf_prob = bf_mu + bf_sigma * rng.standard_normal()
        bf_prob = 0.0 if bf_prob < 0.0 else bf_prob
        
        one_q_depo_prob = one_q_depo_mu + one_q_depo_sigma * rng.standard_normal()
        one_q_depo_prob = 0.0 if one_q_depo_prob < 0.0 else one_q_depo_prob
        
        m.add_noise(BitFlip(z_bf_prob), ObservableCriteria(observables=Observable.Z, qubits=qi))
        #m.add_noise(BitFlip(bf_prob), ObservableCriteria(qubits=qi))
        
        m.add_noise(Depolarizing(one_q_depo_prob), GateCriteria(qubits=qi))
        for qj in range(11):
            if not qj == qi:
                two_q_depo_prob = two_q_depo_mu + two_q_depo_sigma * rng.standard_normal()
                two_q_depo_prob = 0.0 if two_q_depo_prob < 0.0 else two_q_depo_prob
                
                m.add_noise(TwoQubitDepolarizing(two_q_depo_prob), GateCriteria(gates=[Gate.CNot, Gate.Swap, Gate.CPhaseShift], qubits=[qi, qj]))
    return m

# build my circuit here
c = Circuit().h(0).cnot(0,1)
# SOME GATES GET APPLIED

# examine the noiseless circuit 
print(c)

# apply the noise model to the circuit 
nm = noise_model()
c = nm.apply(c)

# examine the noisy circuit 
print(c)

# run the simulation!
device = LocalSimulator('braket_dm')
result = device.run(c, shots=1000).result()
measurement = result.measurement_counts
print('measurement results:', measurement)

# # now improve the mapping based on the results!

# # build a simple circuit
# circ = Circuit().h(0).cnot(0,1)

# # define a noise channel
# noise = noises.BitFlip(probability=0.5)

# # add noise to every gate in the circuit
# circ.apply_gate_noise(noise)

# # select the local noise simulator
# device = LocalSimulator('braket_dm')

# # run the circuit on the local simulator
# task = device.run(circ, shots = 1000)

# # visualize the results
# result = task.result()
# measurement = result.measurement_counts
# print('measurement results:', measurement)

T  : |0|1|
          
q0 : -H-C-
        | 
q1 : ---X-

T  : |0|1|
T  : |      0       |      1      |
                                   
q0 : -H-DEPO(0.0021)-C-DEPO(0.065)-
                     | |           
q1 : ----------------X-DEPO(0.065)-

T  : |      0       |      1      |
measurement results: Counter({'00': 490, '11': 484, '01': 14, '10': 12})


## Trying random circuits

In [28]:
nm = noise_model()

In [81]:
cnot_fidelities = [[0 for j in range(11)] for i in range(11)]

for i in range(0,10):
    for j in range(i+1,11):
        c = Circuit()
        for k in range(0,11):
            c.i(k)

        # build my circuit here
        c.h(i)
        c.cnot(i,j)
        # SOME GATES GET APPLIED

        # examine the noiseless circuit 
        # print(c)

        # apply the noise model to the circuit 
        
        c = nm.apply(c)

        # examine the noisy circuit 
        #print(c)

        # run the simulation!
        device = LocalSimulator('braket_dm')
        result = device.run(c, shots=1000).result()
        measurement = result.measurement_counts
        
        def getString(i,j):
            base = '00000000000'
            return base[0:i] + '1' + base[i+1:j]+'1'+base[j+1:]
        
        fidelity = (measurement['00000000000'] + measurement[getString(i,j)])/1000
        cnot_fidelities[i][j] = fidelity
        
        #print('measurement results for ' + str(i) + ',' + str(j) + ':' + str(fidelity) + '  details: ')
        
for row in fidelities:
    print(row)

[[0, 0.961, 0.955, 0.961, 0.957, 0.956, 0.958, 0.953, 0.947, 0.948, 0.95], [0, 0, 0.948, 0.941, 0.948, 0.944, 0.954, 0.954, 0.951, 0.938, 0.948], [0, 0, 0, 0.961, 0.93, 0.951, 0.952, 0.945, 0.954, 0.941, 0.943], [0, 0, 0, 0, 0.947, 0.946, 0.943, 0.946, 0.948, 0.957, 0.954], [0, 0, 0, 0, 0, 0.942, 0.949, 0.954, 0.968, 0.961, 0.948], [0, 0, 0, 0, 0, 0, 0.951, 0.959, 0.95, 0.959, 0.962], [0, 0, 0, 0, 0, 0, 0, 0.951, 0.959, 0.953, 0.956], [0, 0, 0, 0, 0, 0, 0, 0, 0.946, 0.953, 0.953], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0.947, 0.946], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.948], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [82]:
for row in fidelities:
    print(row)

[0, 0.961, 0.955, 0.961, 0.957, 0.956, 0.958, 0.953, 0.947, 0.948, 0.95]
[0, 0, 0.948, 0.941, 0.948, 0.944, 0.954, 0.954, 0.951, 0.938, 0.948]
[0, 0, 0, 0.961, 0.93, 0.951, 0.952, 0.945, 0.954, 0.941, 0.943]
[0, 0, 0, 0, 0.947, 0.946, 0.943, 0.946, 0.948, 0.957, 0.954]
[0, 0, 0, 0, 0, 0.942, 0.949, 0.954, 0.968, 0.961, 0.948]
[0, 0, 0, 0, 0, 0, 0.951, 0.959, 0.95, 0.959, 0.962]
[0, 0, 0, 0, 0, 0, 0, 0.951, 0.959, 0.953, 0.956]
[0, 0, 0, 0, 0, 0, 0, 0, 0.946, 0.953, 0.953]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.947, 0.946]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.948]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [86]:
swap_fidelities = [[0 for j in range(11)] for i in range(11)]

for i in range(0,10):
    for j in range(i+1,11):
        c = Circuit()
        for k in range(0,11):
            c.i(k)

        # build my circuit here
        c.h(i)
        c.swap(i,j)
        # SOME GATES GET APPLIED

        # examine the noiseless circuit 
        # print(c)

        # apply the noise model to the circuit 
        
        c = nm.apply(c)

        # examine the noisy circuit 
        #print(c)

        # run the simulation!
        device = LocalSimulator('braket_dm')
        result = device.run(c, shots=1000).result()
        measurement = result.measurement_counts
        
        def getString(i,j):
            base = '00000000000'
            return base[0:j]+'1'+base[j+1:]
        
        fidelity = (measurement['00000000000'] + measurement[getString(i,j)])/1000
        swap_fidelities[i][j] = fidelity

In [87]:
for row in swap_fidelities:
    print(row)

[0, 0.958, 0.952, 0.956, 0.959, 0.955, 0.945, 0.961, 0.955, 0.962, 0.939]
[0, 0, 0.948, 0.945, 0.943, 0.948, 0.947, 0.958, 0.947, 0.957, 0.942]
[0, 0, 0, 0.957, 0.938, 0.946, 0.948, 0.943, 0.95, 0.957, 0.959]
[0, 0, 0, 0, 0.954, 0.948, 0.954, 0.958, 0.947, 0.956, 0.956]
[0, 0, 0, 0, 0, 0.943, 0.937, 0.938, 0.95, 0.945, 0.93]
[0, 0, 0, 0, 0, 0, 0.949, 0.946, 0.946, 0.943, 0.951]
[0, 0, 0, 0, 0, 0, 0, 0.942, 0.965, 0.957, 0.95]
[0, 0, 0, 0, 0, 0, 0, 0, 0.941, 0.949, 0.961]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.95, 0.94]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.945]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [5]:
def qft_circuit(n):
    circuit = Circuit()
    
    for i in range(0, n):
        circuit.h(i)
        for j in range(1, n - i):
            circuit.cphaseshift(i+j,i,(2 * np.pi / math.pow(2,j)))

    return circuit
    

In [6]:
qft = qft_circuit(4)

print(qft)


T  : |0|     1     |      2      |           3           |      4      |     5     |6|
                                                                                      
q0 : -H-PHASE(3.14)-PHASE(1.57)---PHASE(0.79)-----------------------------------------
        |           |             |                                                   
q1 : ---C-----------|-----------H-|-----------PHASE(3.14)-PHASE(1.57)-----------------
                    |             |           |           |                           
q2 : ---------------C-------------|-----------C-----------|-----------H-PHASE(3.14)---
                                  |                       |             |             
q3 : -----------------------------C-----------------------C-------------C-----------H-

T  : |0|     1     |      2      |           3           |      4      |     5     |6|


In [8]:
for row in qft.instructions:
    print(row)
    

Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Instruction('operator': CPhaseShift('angle': 3.141592653589793, 'qubit_count': 2), 'target': QubitSet([Qubit(1), Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Instruction('operator': CPhaseShift('angle': 1.5707963267948966, 'qubit_count': 2), 'target': QubitSet([Qubit(2), Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Instruction('operator': CPhaseShift('angle': 0.7853981633974483, 'qubit_count': 2), 'target': QubitSet([Qubit(3), Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Instruction('operator': CPhaseShift('angle': 3.141592653589793, 'qubit_count': 2), 'target': QubitSet([Qubit(2), Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power':

In [4]:
def repeat_h():
    circuit= Circuit()
    
    circuit.h(0)
    circuit.h(0)
    circuit.h(0)
    circuit.x(1)
    circuit.h(1)
    circuit.x(1)
    circuit.y(2)
    circuit.y(2)
    circuit.z(3)
    
    return circuit

for row in repeat_h().instructions:
    print(row)

Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Instruction('operator': X('qubit_count': 1), 'target': QubitSet([Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Instruction('operator': X('qubit_count': 1), 'target': QubitSet([Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Instruction('operator': Y('qubit_count': 1), 'target': QubitSet([Qubit(2)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)
Instruction('operator': Y('qubit_count': 1), 'target': 

In [16]:
print(repeat_h().instructions)

[Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': X('qubit_count': 1), 'target': QubitSet([Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': X('qubit_count': 1), 'target': QubitSet([Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': Y('qubit_count': 1), 'target': QubitSet([Qubit(2)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': Y('qubit_count': 1), 't

In [19]:
print(repeat_h().instructions[0])

Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)


In [6]:
def convert_instructions(instructions, n):
    # Find the maximum qubit index to determine the size of the output array
    max_qubit_index = n
    # Initialize the output array with empty lists for each qubit
    qubit_operations = [[] for _ in range(max_qubit_index)]
    # Iterate through instructions and append operations to the corresponding qubit
    for i in range(0, len(instructions)):
        inst = instructions[i]
        if(len(inst.target) > 1):
            qubit_operations[inst.target[0]].append({'operator': inst.operator.name, 'time' : i, 'first': True })
            qubit_operations[inst.target[1]].append({'operator': inst.operator.name, 'time' : i, 'first': False})
        else:
            qubit_operations[inst.target[0]].append({'operator': inst.operator.name, 'time': i})
                
        # Additional processing for control-target operations could be added here if necessary
    return qubit_operations

In [7]:
repeat_instruct_arr = convert_instructions(repeat_h().instructions,4)
print(repeat_instruct_arr)

[[{'operator': 'H', 'time': 0}, {'operator': 'H', 'time': 1}, {'operator': 'H', 'time': 2}], [{'operator': 'X', 'time': 3}, {'operator': 'H', 'time': 4}, {'operator': 'X', 'time': 5}], [{'operator': 'Y', 'time': 6}, {'operator': 'Y', 'time': 7}], [{'operator': 'Z', 'time': 8}]]


In [54]:
print(convert_instructions(qft_circuit(4).instructions,4))

[[{'operator': 'H', 'time': 0}, {'operator': 'CPhaseShift', 'time': 1, 'first': False}, {'operator': 'CPhaseShift', 'time': 2, 'first': False}, {'operator': 'CPhaseShift', 'time': 3, 'first': False}], [{'operator': 'CPhaseShift', 'time': 1, 'first': True}, {'operator': 'H', 'time': 4}, {'operator': 'CPhaseShift', 'time': 5, 'first': False}, {'operator': 'CPhaseShift', 'time': 6, 'first': False}], [{'operator': 'CPhaseShift', 'time': 2, 'first': True}, {'operator': 'CPhaseShift', 'time': 5, 'first': True}, {'operator': 'H', 'time': 7}, {'operator': 'CPhaseShift', 'time': 8, 'first': False}], [{'operator': 'CPhaseShift', 'time': 3, 'first': True}, {'operator': 'CPhaseShift', 'time': 6, 'first': True}, {'operator': 'CPhaseShift', 'time': 8, 'first': True}, {'operator': 'H', 'time': 9}]]


In [130]:
def gate_cancellation(instruct_arr):
    for gateLine in instruct_arr:
        i = 1
        while(i < len(gateLine)):
            print(gateLine[i])
            a = gateLine[i]['operator']
            b = gateLine[i-1]['operator']
            if(a == b and (a == 'H' or a == 'X' or a == 'Y' or a == 'Z')):
                del gateLine[i]
                del gateLine[i-1]
            i += 1
            

In [9]:
gate_cancellation(repeat_instruct_arr)
print(repeat_instruct_arr)

[{'operator': 'H', 'time': 0}, {'operator': 'H', 'time': 1}, {'operator': 'H', 'time': 2}]
{'operator': 'H', 'time': 1}
[{'operator': 'X', 'time': 3}, {'operator': 'H', 'time': 4}, {'operator': 'X', 'time': 5}]
{'operator': 'H', 'time': 4}
{'operator': 'X', 'time': 5}
[{'operator': 'Y', 'time': 6}, {'operator': 'Y', 'time': 7}]
{'operator': 'Y', 'time': 7}
[{'operator': 'Z', 'time': 8}]
[[{'operator': 'H', 'time': 2}], [{'operator': 'X', 'time': 3}, {'operator': 'H', 'time': 4}, {'operator': 'X', 'time': 5}], [], [{'operator': 'Z', 'time': 8}]]


In [133]:
def convert_instructions(instructions, n):
    """
    converts the braket base instructions file into a form that is easier to work with
    """
    
    max_qubit_index = n
    qubit_operations = [[] for _ in range(max_qubit_index)]
    
    for i in range(0, len(instructions)):
        inst = instructions[i]
        if(len(inst.target) > 1):
            qubit_operations[inst.target[0]].append({'operator': inst.operator.name, 'time' : i, 'first': True })
            qubit_operations[inst.target[1]].append({'operator': inst.operator.name, 'time' : i, 'first': False})
        else:
            qubit_operations[inst.target[0]].append({'operator': inst.operator.name, 'time': i})
                
    return qubit_operations

def gate_cancellation(instruct_arr):
    """
    Goes through truncated form of circuit and removes operators which square to the identity.
    """
    for gateLine in instruct_arr:
        i = 1
        while(i < len(gateLine)):
            print(gateLine[i])
            a = gateLine[i]['operator']
            b = gateLine[i-1]['operator']
            if(a == b and (a == 'H' or a == 'X' or a == 'Y' or a == 'Z')):
                del gateLine[i]
                del gateLine[i-1]
            i += 1
            

def gate_cancelled_circuit(circuit,n):
    """
    Takes in the circuit and then removes any duplicate graphs.
    * Put in the parameter of number of total qubits along with the circuit
    """
    
    instructs = convert_instructions(circuit.instructions,n)
    gate_cancellation(instructs)
    
    new_circuit = Circuit()
    
    for i in range(0, n):
        circuit.i(i)
    
    print(circuit.instructions)
    
    for i in range(len(circuit.instructions)):
        row = circuit.instructions[i]

        op_name = row.operator.name
        
        target = row.target
        
        if(len(instructs[target[0]]) == 0 or instructs[target[0]][0]['time'] != i):
            continue

        if op_name == "H":
            new_circuit.h(target)
        elif op_name == "I":
            new_circuit.i(target)
        elif op_name == "X":
            new_circuit.x(target)
        elif op_name == "Y":
            new_circuit.y(target)
        elif op_name == "Z":
            new_circuit.z(target)
        elif op_name == "S":
            new_circuit.s(target)
        elif op_name == "Si":
            new_circuit.si(target)
        elif op_name == "T":
            new_circuit.t(target)
        elif op_name == "Ti":
            new_circuit.ti(target)
        elif op_name == "V":
            new_circuit.v(target)
        elif op_name == "V":
            new_circuit.vi(target)
        elif op_name == "Rx":
            new_circuit.rx(target, -row.operator.anglegle)
        elif op_name == "Ry":
            new_circuit.ry(target, -row.operator.angle)
        elif op_name == "Rz":
            new_circuit.rz(target, -row.operator.angle)
        elif op_name == "PhaseShift":
            new_circuit.phaseshift(target, -row.operator.angle)
        elif op_name == "CNot":
            new_circuit.cnot(*target)
        elif op_name == "Swap":
            new_circuit.swap(*target)
        elif op_name == "ISwap":
            new_circuit.iswap(*target, -np.pi / 2)
        elif op_name == "PSwap":
            new_circuit.pswap(*target, -row.operator.angle)
        elif op_name == "XY":
            new_circuit.xy(*target, -row.operator.angle)
        elif op_name == "CPhaseShift":
            new_circuit.cphaseshift(*target, -row.operator.angle)
        elif op_name == "CPhaseShift00":
            new_circuit.cphaseshift00(*target, -row.operator.angle)
        elif op_name == "CPhaseShift01":
            new_circuit.cphaseshift01(*target, -row.operator.angle)
        elif op_name == "CPhaseShift10":
            new_circuit.cphaseshift10(*target, -row.operator.angle)
        elif op_name == "CY":
            new_circuit.cy(*target)
        elif op_name == "CZ":
            new_circuit.cz(*target)
        elif op_name == "XX":
            new_circuit.xx(*target, -row.operator.angle)
        elif op_name == "YY":
            new_circuit.yy(*target, -row.operator.angle)
        elif op_name == "ZZ":
            new_circuit.zz(*target, -row.operator.angle)
        elif op_name == "CCNot":
            new_circuit.ccnot(*target)
        elif op_name == "CSwap":
            new_circuit.cswap(*target)
            
        del instructs[target[0]][0]           
    return new_circuit
        


In [135]:
bruh = repeat_h()

print(bruh)
new = gate_cancelled_circuit(bruh,4)
print(new)

T  : |0|1|2|
            
q0 : -H-H-H-
            
q1 : -X-H-X-
            
q2 : -Y-Y---
            
q3 : -Z-----

T  : |0|1|2|
{'operator': 'H', 'time': 1}
{'operator': 'H', 'time': 4}
{'operator': 'X', 'time': 5}
{'operator': 'Y', 'time': 7}
[Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': X('qubit_count': 1), 'target': QubitSet([Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': X('qubit_count': 1), 'target': QubitSet([Qubit(1

In [83]:
rand_circuit = Circuit()
rand_circuit.h(0)
rand_circuit.cnot(0,1)

print(rand_circuit)

T  : |0|1|
          
q0 : -H-C-
        | 
q1 : ---X-

T  : |0|1|


In [84]:
base_instructs = convert_instructions(rand_circuit.instructions,2)

print(base_instructs)
print(rand_circuit.instructions)

[[{'operator': 'H', 'time': 0}, {'operator': 'CNot', 'time': 1, 'first': True}], [{'operator': 'CNot', 'time': 1, 'first': False}]]
[Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(0), Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)]


## Testing the sequential

In [115]:
c = Circuit()

c.h(0)
c.cnot(0,1)
c.cnot(1,2)

c = nm.apply(c)

device = LocalSimulator('braket_dm')
result = device.run(c, shots=1000).result()
measurement = result.measurement_counts

print(measurement)

Counter({'000': 472, '111': 440, '011': 29, '100': 27, '001': 9, '110': 9, '010': 8, '101': 6})


## Testing Total Fidelity?

In [17]:
import sys
sys.path.insert(0, './circuits')  # Adjust the path if necessary.

In [18]:
import circuits.qft

ModuleNotFoundError: No module named 'circuits'

In [23]:
def qft(n, arr):
    circuit = Circuit()
    
    for i in range(len(arr)):
        if(arr[i] == 1):
            circuit.x(i)
    
    for i in range(0, n):
        circuit.h(i)
        for j in range(1, n - i):
            circuit.cphaseshift(i+j,i,(2 * np.pi / math.pow(2,j)))

    return circuit

In [85]:
qft_circuit = qft(3, [0,0,1])
qft_no_noise = qft(3,[0,0,1])

print(qft_circuit)
print(qft_no_noise)

T  : |0|     1     |      2      |     3     |4|
                                                
q0 : -H-PHASE(3.14)-PHASE(1.57)-----------------
        |           |                           
q1 : ---C-----------|-----------H-PHASE(3.14)---
                    |             |             
q2 : -X-------------C-------------C-----------H-

T  : |0|     1     |      2      |     3     |4|
T  : |0|     1     |      2      |     3     |4|
                                                
q0 : -H-PHASE(3.14)-PHASE(1.57)-----------------
        |           |                           
q1 : ---C-----------|-----------H-PHASE(3.14)---
                    |             |             
q2 : -X-------------C-------------C-----------H-

T  : |0|     1     |      2      |     3     |4|


In [100]:
qft_circuit = nm.apply(qft_circuit)

result_no_noise = device.run(qft_circuit, shots=1000).result()
print(result_no_noise.measurement_counts)

Counter({'111': 140, '101': 134, '110': 131, '100': 124, '011': 122, '010': 121, '001': 117, '000': 111})


In [87]:
result_noise = device.run(qft_no_noise, shots=1000).result()
print(result_noise.measurement_counts)

Counter({'011': 131, '000': 129, '101': 128, '111': 125, '010': 124, '001': 124, '110': 124, '100': 115})


In [ ]:
def conv_to_vector(measurement):

    keys_sorted = sorted(measurement.keys())
    values_sorted = [measurement[key] for key in keys_sorted]

    # Convert to numpy vector
    vector = np.array(values_sorted)

    return vector


def compare_results(noisy, real):
    ans = 0
    for key in noisy:
        if key in real:
            ans += noisy[key]*real[key]

    vec_n = conv_to_vector(noisy)
    vec_r = conv_to_vector(real)
    
    ans /= np.linalg.norm(vec_n)*np.linalg.norm(vec_r)
    
    return ans